In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import matplotlib.pyplot as plt
import os

2025-08-12 16:25:30.953844: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-12 16:25:31.310302: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-12 16:25:31.474161: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755012331.743527    8002 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755012331.825440    8002 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755012332.318617    8002 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
# ===== Configuration =====
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
# Model parameters
IMG_SHAPE = (224, 224, 1)
BATCH_SIZE = 32
EPOCHS = 25
MODEL_DIR = "saved_models"
os.makedirs(MODEL_DIR, exist_ok=True)

In [4]:
# ===== Load Preprocessed Data =====
data = np.load('preprocessed_data/preprocessed_images.npz')

# Add channel dimension (required for CNN)
def add_channel(X):
    return X[..., np.newaxis]

X_combined = add_channel(data['X_combined'])
y_combined = data['y_combined']

In [5]:
def build_cnn_model():
    model = Sequential([
        # Convolutional layers
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=IMG_SHAPE),
        BatchNormalization(),
        MaxPooling2D((2,2)),
        
        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2)),
        
        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2)),
        
        Conv2D(256, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2,2)),
        
        # Classification head
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001),
                    loss='binary_crossentropy',
                    metrics=['accuracy', 
                            tf.keras.metrics.Precision(name='precision'),
                            tf.keras.metrics.Recall(name='recall')])
    return model

In [6]:
# ===== Train & Evaluate Model =====
def train_evaluate_model(X, y, dataset_name):
    print(f"\n{'='*50}")
    print(f"Training Model for: {dataset_name.upper()} Dataset")
    print(f"{'='*50}")
    
    # 1. Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.15, stratify=y, random_state=SEED
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.176, stratify=y_train, random_state=SEED
    )  # 70/15/15 split
    
    print(f"Train: {X_train.shape[0]} samples | Val: {X_val.shape[0]} | Test: {X_test.shape[0]}")
    
    # 2. Compute class weights
    class_counts = np.bincount(y_train.astype(int))
    class_weights = {0: sum(class_counts)/class_counts[0], 
                        1: sum(class_counts)/class_counts[1]}
    print(f"Class weights: {class_weights}")
    
    # 3. Build model
    model = build_cnn_model()
    
    # 4. Callbacks
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=1e-7),
        ModelCheckpoint(os.path.join(MODEL_DIR, f"best_{dataset_name}_model.keras"), 
                        save_best_only=True)
    ]
    
    # 5. Train model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        class_weight=class_weights,
        callbacks=callbacks,
        verbose=1
    )
    
    # 6. Evaluate on test set
    print("\nTest Set Evaluation:")
    test_results = model.evaluate(X_test, y_test, verbose=0)
    print(f"Loss: {test_results[0]:.4f} | Accuracy: {test_results[1]:.4f}")
    print(f"Precision: {test_results[2]:.4f} | Recall: {test_results[3]:.4f}")
    
    # Predictions for additional metrics
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    print(classification_report(y_test, y_pred, target_names=['Non-Cancerous', 'Malignant']))
    
    # ROC-AUC
    roc_auc = roc_auc_score(y_test, model.predict(X_test))
    print(f"ROC-AUC: {roc_auc:.4f}")
    
    # 7. Plot training history
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Loss Evolution')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title('Accuracy Evolution')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(f"plots/{dataset_name}_training_history.png")
    plt.close()
    
    return model, test_results


In [7]:
model = build_cnn_model()
model.summary()

/home/mohammedah10/miniconda3/envs/deeplearning_env/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-08-12 16:25:56.845781: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    25,690,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,080,897 (99.49 MB)

 Trainable params: 26,079,937 (99.49 MB)

 Non-trainable params: 960 (3.75 KB)

In [8]:
# ===== Train Model =====
combined_model, combined_results = train_evaluate_model(X_combined, y_combined, "combined")



Training Model for: COMBINED Dataset
Train: 831 samples | Val: 178 | Test: 179
Class weights: {0: 1.6787878787878787, 1: 2.4732142857142856}


/home/mohammedah10/miniconda3/envs/deeplearning_env/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25


2025-08-12 16:26:01.965152: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 166785024 exceeds 10% of free system memory.
2025-08-12 16:26:22.942223: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2025-08-12 16:26:23.719907: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2025-08-12 16:26:24.351448: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2025-08-12 16:26:38.155505: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


26/26 ━━━━━━━━━━━━━━━━━━━━ 414s 15s/step - accuracy: 0.5396 - loss: 33.5161 - precision: 0.4807 - recall: 0.5290 - val_accuracy: 0.4045 - val_loss: 4.2288 - val_precision: 0.4045 - val_recall: 1.0000 - learning_rate: 0.0010
Epoch 2/25
26/26 ━━━━━━━━━━━━━━━━━━━━ 388s 15s/step - accuracy: 0.6104 - loss: 14.2752 - precision: 0.5566 - recall: 0.6185 - val_accuracy: 0.4213 - val_loss: 4.2507 - val_precision: 0.4114 - val_recall: 1.0000 - learning_rate: 0.0010
Epoch 3/25
26/26 ━━━━━━━━━━━━━━━━━━━━ 381s 15s/step - accuracy: 0.5974 - loss: 8.1866 - precision: 0.5421 - recall: 0.6110 - val_accuracy: 0.5955 - val_loss: 3.4732 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 4/25
26/26 ━━━━━━━━━━━━━━━━━━━━ 410s 16s/step - accuracy: 0.6216 - loss: 2.4391 - precision: 0.5549 - recall: 0.7742 - val_accuracy: 0.5955 - val_loss: 3.6680 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 5/25
26/26 ━━━━━━━━━━━━━━━━━━━━ 413s 16s/step - acc

In [11]:
combined_model.save(os.path.join(MODEL_DIR, "final_combined_model.keras"))

In [10]:
# ===== Compare Model Performance =====
print("\nModel Performance Comparison:")
print(f"{'Dataset':<10} | {'Accuracy':<8} | {'Precision':<8} | {'Recall':<8} | {'ROC-AUC':<8}")
print("-"*50)
print(f"{'Combined':<10} | {combined_results[1]:.4f}    | {combined_results[2]:.4f}    | {combined_results[3]:.4f}    | {roc_auc_score(y_combined_test, combined_model.predict(X_combined_test)):.4f}")


Model Performance Comparison:
Dataset    | Accuracy | Precision | Recall   | ROC-AUC 
--------------------------------------------------


NameError: name 'y_combined_test' is not defined